# Init

In [1]:
try: 
    import os
    import glob
    import sys
    import math
    from typing import List, Optional
    from functools import partial
    import itertools
    import copy
except Exception as e:
    print(e)
    print("Some of the libraries needed to run this script were not installed or were not loaded. Please install the libraries before proceeding.")

In [2]:
sys.path.append(os.environ['DEV_AUTOTS'])
sys.path.append(os.environ['CAPSTONE_PYTHON_SOURCE'])
folder = os.environ['CAPSTONE_DATA']

In [3]:
try:
    # Data Tables
    import pandas as pd
    import numpy as np

    # Plotting
    import matplotlib.pyplot as plt
    import plotly.offline as py
    from plotly.offline import plot
    py.init_notebook_mode(connected=True)

    # EDA and Feature Engineering
    from scipy.spatial.distance import euclidean, pdist, squareform
    import statsmodels.api as sm

    # Auto Time Series
    import auto_ts as AT

    # Optimizer
    from skopt import gp_minimize
    from skopt.space import Real, Integer
    from skopt.plots import plot_convergence
except Exception as e:
    print(e)
    print("Some of the libraries needed to run this script were not installed or were not loaded. Please install the libraries before proceeding.")

Running Auto Timeseries version: 0.0.24


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
try:
    from ETL.ETL import loadDataset, getTopProducts
    from similarity.similarity import mergeTopSimilar, loadSimilarity
    from charting.charting import surface3DChart
except Exception as e:
    print(e)
    print("Some of the libraries needed to run this script were not installed or were not loaded. Please install the libraries before proceeding.")    

In [6]:

dataRaw= loadDataset(version=4)


# Prep Data

In [7]:
#Parameters
#ChainMaster = 'SPECS'
#ProdCat='SUP PREM WHISKEY'
TOP_PRODUCTS = 3  # How many products to consider in the category
TOP_SIMILAR = 1  # Get TOP_SIMILAR most similar products

LOG_TRANSFORM = True # Take log of 9L cases to smooth out peaks and valleys
ZERO_ADDER = 0.1 

RESAMPLE_FREQ = 'M'

# Pricing changes every 4 weeks
if RESAMPLE_FREQ == 'M':
    FORECAST_PERIOD = 1
if RESAMPLE_FREQ == 'W':
    FORECAST_PERIOD = 4 
if RESAMPLE_FREQ == '2W':
    FORECAST_PERIOD = 2 

# Seasonal Period
if RESAMPLE_FREQ == 'M':
    SEASONAL_PERIOD = 12  # Yearly
if RESAMPLE_FREQ == 'W':
    SEASONAL_PERIOD = 13 # Quarterly (we can also take yearly = 52, but SARIMAX becomes too slow)
if RESAMPLE_FREQ == '2W':
    SEASONAL_PERIOD = 13 # This becomes problematic --> for quarterly, should we take 6 biweekly periods or 7 bi-weekly periods. Instead I just took half yearly period  

print("="*50)
print("Parameters being used...")
print("="*50)
print(f"Resample Frequency = {RESAMPLE_FREQ}")
print(f"Forecast Period = {FORECAST_PERIOD}")
print(f"Seasonal Period = {SEASONAL_PERIOD}")


Parameters being used...
Resample Frequency = M
Forecast Period = 1
Seasonal Period = 12


# Model Flow

## Functions

In [8]:
COL_TIME = 'WeekDate'
COL_PREDS = ['9L Cases'] #Demand
COL_PRICE= ['Dollar Sales per 9L Case'] #Price

def modelsLoadData(ProductsList,dataRaw,ChainMaster):
    all_data = []
    
    if(ChainMaster!=''):
        dfSimilarity = loadSimilarity(version=4)
    else:
        dfSimilarity = loadSimilarity(version=4,allCustomers=True)
    
    for i, Product in enumerate(ProductsList):
        (dataModel,colExog,colEnc,colDec) = mergeTopSimilar(dataRaw, dfSimilarity
                                                            ,ChainMaster=ChainMaster
                                                            ,Product=Product
                                                            ,ProductsList=ProductsList
                                                            ,topn=TOP_SIMILAR 
                                                            ,periodCol = COL_TIME
                                                            ,resampleFreq=RESAMPLE_FREQ
                                                            ,encodeCols=True)


        if i == 0: print(f"Decoder: {colDec}")

        print("\n\n")
        print("-"*50)
        print(f"Product: {colDec.get(str(i))}")
        print("-"*50)

        #colExog = colExog + colEndog
        print(f"Exogenous Price Columns: {colExog}")

        allCols=[COL_TIME]+COL_PREDS+ colExog
        data=dataModel[allCols]
        print(f"% of weeks without a purchase: {sum(data['9L Cases'] == 0)/data.shape[0]*100}")
        all_data.append(data)
    
    all_data_non_transformed =  copy.deepcopy(all_data)
    
    if LOG_TRANSFORM: 
        print("Log Transforming")
        for i in np.arange(len(all_data)):
            all_data_non_transformed[i] = all_data[i].copy(deep=True)
            all_data[i][COL_PREDS] = np.log10(all_data[i][COL_PREDS] + ZERO_ADDER)
            print(f"\tProduct: {colDec.get(str(i))}")
    return(all_data,all_data_non_transformed,colExog,colEnc,colDec)
            
def ModelsWhiteNoise(all_data)           :
    ## WHITE NOISE TEST
    white_noise_all = []
    white_noise_df_all = []
    #check if there are 12, 24, 48 data points
    for i, data in enumerate(all_data):
        lags=[12,24,48]
        lags=[x  for x in lags if x < data.shape[0]]
        white_noise_df = sm.stats.acorr_ljungbox(data[COL_PREDS], lags=lags, return_df=True)
        white_noise_df_all.append(white_noise_df)
        if any(white_noise_df['lb_pvalue'] > 0.05):
            white_noise = True
        else:
            white_noise = False
        white_noise_all.append(white_noise)

        print(white_noise_df)
        print(f"\nIs Data White Noise: {white_noise}")
    
    return(white_noise_all)  

def ModelsTestTrain(all_data,all_data_non_transformed):
    all_train = []
    all_test = []

    all_train_non_transformed = []
    all_test_non_transformed = []

    for i, data in enumerate(all_data):
        train = all_data_non_transformed[i].iloc[:-FORECAST_PERIOD]
        test = all_data_non_transformed[i].iloc[-FORECAST_PERIOD:]
        all_train_non_transformed.append(train)
        all_test_non_transformed.append(test)

        train = data.iloc[:-FORECAST_PERIOD]
        test = data.iloc[-FORECAST_PERIOD:]
        all_train.append(train)
        all_test.append(test)

        print(train.shape,test.shape)
    return(all_train,all_test)

def ModelsFit(all_data,all_train,all_test,withSimilar):
    from joblib import Parallel, delayed
    
    def modelsFun(i):
        train = all_train[i]
        test = all_test[i]
        if(withSimilar==False):
            train = train[train.columns[0:3]] #3rd col has the curr product price

        automl_model = AT.AutoTimeSeries(
            score_type='rmse', forecast_period=FORECAST_PERIOD, # time_interval='Week',
            non_seasonal_pdq=None, seasonality=True, seasonal_period=SEASONAL_PERIOD,
            model_type=['SARIMAX','ML','prophet','auto_SARIMAX'],
            verbose=0)

        automl_model.fit(train, COL_TIME, COL_PREDS, cv=10, sep=',') #cv=10
        return(automl_model)
    
    args = np.arange(len(all_data))
    
    all_models = Parallel(n_jobs=-1, verbose=1
                          #, backend="threading"
                           , backend="loky"
                         )(
             map(delayed(modelsFun), args))
    
    
    return(all_models)

def get_rmse(predictions, targets):
    return np.sqrt(((np.array(predictions) - np.array(targets)) ** 2).mean())

def modelNaive(all_data,all_train,all_test,season=12,windowLength=8):
    from sktime.forecasting.naive import NaiveForecaster
    import statistics 
    from tscv import GapWalkForward # type: ignore
    all_naives=pd.DataFrame(columns=['ID','Best Type','Best RMSE'])
    types=['last','seasonal_last','mean']
    #add window code
    
    NFOLDS=5
    for i, data in enumerate(all_data):
        yTrain = pd.Series(all_train[i][COL_PREDS[0]])
        yTest = pd.Series(all_test[i][COL_PREDS[0]])
        yTrain = yTrain.append(yTest) # merging as we are gong to do cv
        rmses=[]
        for t in types:
            #naive_forecaster = NaiveForecaster(strategy="last")
            cv = GapWalkForward(n_splits=10, gap_size=0, test_size=FORECAST_PERIOD)
            cvRmse=[]
            for fold_number, (train, test) in enumerate(cv.split(yTrain)):
                cv_train = yTrain.iloc[train]
                cv_test = yTrain.iloc[test]
                
                naive_forecaster = NaiveForecaster(strategy=t,sp=season,window_length=windowLength)
                naive_forecaster.fit(cv_train)
                yPred = naive_forecaster.predict(np.arange(len(cv_test)))
                rmse=get_rmse(yPred, cv_test)
                cvRmse.append(rmse)
            rmses.append(np.mean(cvRmse))
        bestRmse = np.argmin(rmses)
        all_naives=all_naives.append(
            {'ID':i
             ,'Best Type' : types[bestRmse]
             ,'Best RMSE':rmses[bestRmse]
             ,'All Types':[types]
             ,'All RMSEs':[rmses]
            }
            ,ignore_index=True)
    print(all_naives)
    return(all_naives) 

def centerLog(text,w,pre='\n',post=''):
    t=int((w-len(text))/2-1)
    return(pre+'='*t+' '+text+' '+'='*(w-len(text)-t-2)+post)

def printLog(main,subs,linesPre=2,linesPost=1):
    import datetime
    if(isinstance(subs,list)== False): subs=[subs]
    maxw=max([len(x) for x in [main] + subs])+10
    print("\n"*linesPre
          +"="*maxw+" ("+str(datetime.datetime.now())+")"
          +centerLog(main,maxw)
          +''.join([centerLog(x,maxw) for x in subs])
          +"\n"+"="*maxw
          +"\n"*linesPost
         )

def runModels(ProductsList,dataRaw,ChainMaster):
    printLog("GET DATA",ChainMaster)
    all_data,all_data_non_transformed,colExog,colEnc,colDec = modelsLoadData(ProductsList,dataRaw,ChainMaster)
    
    printLog("WHITE NOISE",ChainMaster)
    white_noise = ModelsWhiteNoise(all_data)  
    
    printLog("TEST/TRAIN",ChainMaster)
    all_train, all_test = ModelsTestTrain(all_data,all_data_non_transformed)
    
    all_stats = pd.DataFrame()
    all_stats['Product'] = ProductsList
    all_stats['Chain Master'] = ChainMaster
    all_stats['White Noise'] = white_noise
    
    printLog("NAIVE",ChainMaster)
    naive = modelNaive(all_data,all_train,all_test,season=4,windowLength=8)
    all_stats['Naive Best Type'] = [naive.iloc[x]['Best Type'] for x in np.arange(len(all_data)) ]
    all_stats['Noive Best RMSE'] = [naive.iloc[x]['Best RMSE'] for x in np.arange(len(all_data)) ]
    
     
    printLog("Multivar P0",ChainMaster)
    multivarP0 = ModelsFit(all_data,all_train,all_test,withSimilar = False)
    all_stats['P0 Best Model Name'] = [multivarP0[x].get_leaderboard().iloc[0]['name'] for x in np.arange(len(all_data)) ]
    all_stats['P0 Best Model RMSE'] = [multivarP0[x].get_leaderboard().iloc[0]['rmse'] for x in np.arange(len(all_data)) ]
    all_stats['P0 Best Model'] = [multivarP0[x] for x in np.arange(len(all_data)) ]
    
    printLog("Multivar P0+Sim",ChainMaster)
    multivarP0Sim = ModelsFit(all_data,all_train,all_test,withSimilar = True )
    all_stats['P0+Sim Best Model Name'] = [multivarP0Sim[x].get_leaderboard().iloc[0]['name'] for x in np.arange(len(all_data)) ]
    all_stats['P0+Sim Best Model RMSE'] = [multivarP0Sim[x].get_leaderboard().iloc[0]['rmse'] for x in np.arange(len(all_data)) ]
    all_stats['P0+Sim Best Model'] = [multivarP0Sim[x] for x in np.arange(len(all_data)) ]
   
    return(all_stats)


## Loop

In [9]:

ChainMasters =  [''] +  dataRaw['Chain Master'].unique().tolist() 
ProdCats = dataRaw['Category (CatMan)'].unique().tolist()
display(ChainMasters,ProdCats)

['', 'THE BARREL HOUSE', 'WESTERN BEV LIQ TX', 'SPECS']

['ECONOMY VODKA', 'SUP PREM WHISKEY']

In [10]:
full_stats=pd.DataFrame()
ProdCats = ['SUP PREM WHISKEY']
for ProdCat in ProdCats:
    for ChainMaster in ChainMasters:
        printLog("Running ",[ProdCat,ChainMaster])
        ProductsList = getTopProducts(dataRaw, ChainMaster=ChainMaster, ProdCat=ProdCat, topN=TOP_PRODUCTS, timeCol='WeekDate')
        all_stats=runModels(ProductsList,dataRaw,ChainMaster)
        all_stats['Product Category']=ProdCat
        display(all_stats)
        full_stats=full_stats.append(all_stats,ignore_index=True)

printLog("Completed","")
   




========================== (2020-08-05 18:09:31.105023)
======== Running  ========
==== SUP PREM WHISKEY ====
============  ============



================== (2020-08-05 18:09:31.139931)
==== GET DATA ====
========  ========

resampling to  M
Decoder: {'0': 'JACK DANIELS BLK WHSKY  1L', '1': 'JACK DANIELS BLK WHSKY 3PK W/2L COKE 1.75L', '2': 'JACK DANIELS BLK WHSKY  1.75L'}



--------------------------------------------------
Product: JACK DANIELS BLK WHSKY  1L
--------------------------------------------------
Exogenous Price Columns: ['0', '2']
% of weeks without a purchase: 0.0
resampling to  M



--------------------------------------------------
Product: JACK DANIELS BLK WHSKY 3PK W/2L COKE 1.75L
--------------------------------------------------
Exogenous Price Columns: ['1', '2']
% of weeks without a purchase: 63.41463414634146
resampling to  M



--------------------------------------------------
Product: JACK DANIELS BLK WHSKY  1.75L
---------------------------------------

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.




========================= (2020-08-05 18:13:31.504985)
==== Multivar P0+Sim ====
===========  ============



[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.5min finished


,Product,Chain Master,White Noise,Naive Best Type,Noive Best RMSE,P0 Best Model Name,P0 Best Model RMSE,P0 Best Model,P0+Sim Best Model Name,P0+Sim Best Model RMSE,P0+Sim Best Model,Product Category
0,JACK DANIELS BLK WHSKY 1L,,True,mean,0.043169,SARIMAX,0.027899,<auto_ts.AutoTimeSeries object at 0x000001A1DF...,ML,0.030409,<auto_ts.AutoTimeSeries object at 0x000001A1DE...,SUP PREM WHISKEY
1,JACK DANIELS BLK WHSKY 3PK W/2L COKE 1.75L,,False,mean,1.222095,Prophet,0.927468,<auto_ts.AutoTimeSeries object at 0x000001A1DE...,ML,0.859083,<auto_ts.AutoTimeSeries object at 0x000001A1DE...,SUP PREM WHISKEY
2,JACK DANIELS BLK WHSKY 1.75L,,False,last,0.324455,auto_SARIMAX,0.207729,<auto_ts.AutoTimeSeries object at 0x000001A1DE...,auto_SARIMAX,0.229401,<auto_ts.AutoTimeSeries object at 0x000001A1DD...,SUP PREM WHISKEY


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.9min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.




========================= (2020-08-05 18:34:35.444837)
==== Multivar P0+Sim ====
========= SPECS =========



[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.9min finished


,Product,Chain Master,White Noise,Naive Best Type,Noive Best RMSE,P0 Best Model Name,P0 Best Model RMSE,P0 Best Model,P0+Sim Best Model Name,P0+Sim Best Model RMSE,P0+Sim Best Model,Product Category
0,JACK DANIELS BLK WHSKY 1L,SPECS,True,mean,0.060885,SARIMAX,0.037469,<auto_ts.AutoTimeSeries object at 0x000001A1E5...,ML,0.039430,<auto_ts.AutoTimeSeries object at 0x000001A1EF...,SUP PREM WHISKEY
1,JACK DANIELS BLK WHSKY 1.75L,SPECS,True,mean,0.159196,SARIMAX,0.119452,<auto_ts.AutoTimeSeries object at 0x000001A1E5...,SARIMAX,0.137262,<auto_ts.AutoTimeSeries object at 0x000001A1DF...,SUP PREM WHISKEY
2,JACK DANIELS BLK WHSKY 750M,SPECS,True,mean,0.276661,auto_SARIMAX,0.237800,<auto_ts.AutoTimeSeries object at 0x000001A1DE...,SARIMAX,0.246823,<auto_ts.AutoTimeSeries object at 0x000001A1EF...,SUP PREM WHISKEY




=================== (2020-08-05 18:38:29.279518)
==== Completed ====
========  =========



## Print out

In [11]:
full_stats[full_stats.columns.difference(['P0 Best Model','P0+Sim Best Model'],sort=False)]

,Product,Chain Master,White Noise,Naive Best Type,Noive Best RMSE,P0 Best Model Name,P0 Best Model RMSE,P0+Sim Best Model Name,P0+Sim Best Model RMSE,Product Category
0,JACK DANIELS BLK WHSKY 1L,,True,mean,0.043169,SARIMAX,0.027899,ML,0.030409,SUP PREM WHISKEY
1,JACK DANIELS BLK WHSKY 3PK W/2L COKE 1.75L,,False,mean,1.222095,Prophet,0.927468,ML,0.859083,SUP PREM WHISKEY
2,JACK DANIELS BLK WHSKY 1.75L,,False,last,0.324455,auto_SARIMAX,0.207729,auto_SARIMAX,0.229401,SUP PREM WHISKEY
3,JACK DANIELS BLK WHSKY 1L,THE BARREL HOUSE,False,mean,1.149908,auto_SARIMAX,0.712256,ML,0.703530,SUP PREM WHISKEY
4,JACK DANIELS BLK WHSKY 1.75L,THE BARREL HOUSE,True,last,0.805963,ML,0.572234,ML,0.572227,SUP PREM WHISKEY
5,JACK DANIELS BLK WHSKY 750M,THE BARREL HOUSE,True,mean,0.959053,ML,0.747042,ML,0.747042,SUP PREM WHISKEY
6,JACK DANIELS BLK WHSKY 1.75L,WESTERN BEV LIQ TX,False,mean,1.192388,auto_SARIMAX,0.713414,Prophet,0.607270,SUP PREM WHISKEY
7,JACK DANIELS BLK WHSKY 3PK W/2L COKE 1.75L,WESTERN BEV LIQ TX,False,seasonal_last,1.151758,Prophet,1.000953,ML,0.879541,SUP PREM WHISKEY
8,JACK DANIELS BLK WHSKY 6PK W/MEXI COKE 750M,WESTERN BEV LIQ TX,True,seasonal_last,0.885673,auto_SARIMAX,0.714413,auto_SARIMAX,0.700701,SUP PREM WHISKEY
9,JACK DANIELS BLK WHSKY 1L,SPECS,True,mean,0.060885,SARIMAX,0.037469,ML,0.039430,SUP PREM WHISKEY
